### 1. Load data

In [13]:
import os
import pandas as pd
from data_prep import run_data_prep, prep_dataset_for_modelling

In [35]:
#load orignal data
data_set_name = "BastianLewisData"
data_dir = r"C:\Users\peter\Projects\HR_Forecasting\data"
file_path = os.path.join(data_dir,'BastianLewisData.xlsx')
df = pd.read_excel(file_path,sheet_name='Result 1')

In [24]:
#load latest data
data_set_name = "StrevonExport"
file_path = os.path.join(data_dir,'StrevonExport.xlsx')
df = pd.read_excel(file_path,sheet_name='StrevonExport')

In [36]:
#Drop irrelevant variables
var_file_path = os.path.join(data_dir, 'BastianLewisData.xlsx')
df_vars = pd.read_excel(var_file_path,sheet_name='VariableSelection')
var_drop = list(df_vars['Name'][df_vars['Status'] == 'Drop'])

df.drop(labels = var_drop,  axis= 1, inplace = True)

### 2. Select relevant candidates/rows

In [37]:
# alleen 'prioriteit' = 1 houden & geen afwijsBasisGegevens (isnull) 
print('unique prioriteit values:',  df['prioriteit'].unique())
print('unique afwijsBasisGegevens values:',  df['afwijsBasisGegevens'].unique())

df = df[df['prioriteit'] == 1]
df = df[pd.isnull(df['afwijsBasisGegevens'])]
df.drop(labels = ['prioriteit','afwijsBasisGegevens'],  axis= 1, inplace = True)
print(f"number of candidates left: {len(df)}")
print(df['starter'].value_counts())

unique prioriteit values: [ 1. nan  3.  2.  4.]
unique afwijsBasisGegevens values: [nan 'postal_code' 'age' 'drivers_license' 'dutch_language_skill']
number of candidates left: 4129
0    3993
1     136
Name: starter, dtype: int64


In [29]:
df_backup = df.copy()

In [30]:
#df =df_backup.copy()

### 3. clean dataset

In [31]:
supporting_data_dir = r"C:\Users\peter\Projects\HR_Forecasting\supporting_data"

In [32]:
df = run_data_prep(df, supporting_data_dir)

### 4. Clean categorical variables

In [33]:
model_dir = r"C:\Users\peter\Projects\HR_Forecasting\models"
df_final = prep_dataset_for_modelling(df, model_dir, data_set_name)

working on recruiter
555 missings
recategorizing all values that appear less then 10 times to 'other'
replaced the following values with other: Index(['Jurrian Sturm', 'Wim van Ginkel', 'Stavros Tatanidis',
       'Gwyneth van Diermen', 'Fadoua Beghiyl', 'Gideon Den Admirant'],
      dtype='object')
Keeping the following values: Index(['Job van den Berg', 'Jermaine Symons', 'Lennart Huibers',
       'Isabelle Walschots', 'Richard Felix Omondi', 'Wim Van Ginkel',
       'Job Bruinsma'],
      dtype='object')
_________________________
working on utm_source
172 missings
recategorizing all values that appear less then 10 times to 'other'
replaced the following values with other: Index(['8kvl4ikw535', 'l', 'fb-ig-web', 'duckduckgo', 'ecosia', 'linktr',
       'youtube', 'googleadservices', 'com', 'nationale vacature gids', 'm',
       'nl'],
      dtype='object')
Keeping the following values: Index(['google', 'strevon', 'fb-ig', 'facebook', 'indeed', 'undefined',
       'nationaleberoepengi

In [34]:
df_final.to_csv(os.path.join(data_dir,f"{data_set_name}_final.csv"),index=False)